In [ ]:
import tensorflow as tf

from collections import deque

import numpy as np

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import gym

In [ ]:
import wandb
wandb.login()
run=wandb.init(project="lab3", entity="ieor-4575", tags=["tf2"])

## DQN (Deep Q Network)

In previous Labs, we have learned to use Tensorflow to build deep learning models. In this lab, we will apply deep learning as function approximations in reinforcement learning. 

Reference: DQN https://arxiv.org/abs/1312.5602

In tabular Q-learning, we maintain a table of state-action pairs $(s,a)$ and save one action value for each entry $Q(s,a),\forall (s,a)$. At each time step $t$, we are in state $s_t$, then we choose action based on $\epsilon-$greedy strategy. With prob $\epsilon$, choose action uniformly random; with prob $1-\epsilon$, choose action based on $$a_t = \arg\max_a Q(s_t,a)$$ 

We then get the instant reward $r_t$, update the Q-table using the following rule

$$Q(s_t,a_t) \leftarrow (1-\alpha)Q(s_t,a_t) + \alpha (r_t + \max_a \gamma Q(s_{t+1},a))$$

where $\alpha \in (0,1)$ is learning rate. The algorithm is shown to converge in tabular cases. However, in cases where we cannot keep a table for state and action, we need function approximation. Consider using neural network with parameter $\theta$, the network takes as input state $s$ and action $a$. (*there are alternative parameterizations here*). Let $Q_\theta(s,a)$ be the output of the network, to estimate the optimal action value function in state $s$ and take action $a$ (and follow optimal policy thereafter). 

$$Q_\theta(s,a) \approx Q^\ast(s,a)$$

### Bellman optimality equation

We will use Bellman optimality equation to find $\theta$ such that the above approximation holds better. Recall that for optimal Q function $Q^\ast(s,a)$ the following holds for all $(s,a)$

$$Q^\ast(s_t,a_t) = \mathbb{E}\big[r_t + \gamma \max_a Q^\ast(s_{t+1},a)\big]$$

where the expectation is wrt the random reward $r_t$ and transition to the next state $s_{t+1}$. A natural objective to consider is 

$$\min_\theta\  (Q_\theta(s_t,a_t) -\mathbb{E}\big[r_t + \gamma  \max_a  Q_{\hat \theta}(s_{t+1},a)\big])^2$$
at the current or previous $\hat \theta$.



### Building the DQN model

The first step is to build a neural network with parameters $\theta$ that predicts $Q_\theta(s,a)$ for any $(s,a)$. You can either build a network that 
* takes as input a concatenated representation of state and action $(s,a)$ and output one dimensional score $Q_\theta(s,a)$,

or (in case of small number $K$ of discrete actions)

* takes as input a  representation of state $s$ and outputs a $K$-dimensional vector giving scores $Q(s,a), a=1,\ldots, K$ for all actions

Below we have provided a skeleton code (incomplete) for defining and training the Q-function. **You need to fill in the DNN model definition and loss function definition**. Refer to regression lab (lab 2) for help.

In [ ]:
# define neural net Q_\theta(s,a) as a class

class Qfunction(object):
    
    def __init__(self, obssize, actsize, lr):
        """
        obssize: dimension of state space
        actsize: dimension of action space
        sess: sess to execute this Qfunction
        optimizer: 
        """
        # DEFINE THE MODEL
        self.model = tf.keras.models.Sequential([
                    #TODO
                    #input layer
                      #tf.keras.layers.Dense(<input size>, activation='relu') 
                      #intermediate layers
                      #tf.keras.layers.Dense(<intermediate layer size>, activation='relu'),
                      #.....
                      #......
                      #tf.keras.layers.Dense(<output size>)
                ])
        
        # DEFINE THE OPTIMIZER
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        
        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize
        
        # TEST
        self.compute_Qvalues(np.random.randn(obssize).reshape(1, -1), 0)
        self.compute_argmaxQ(np.random.randn(obssize).reshape(1, -1))
    
    def compute_Qvalues(self, states, actions):
        """
        input: list of numsamples state-action pairs
        output: List of Q values for each input (s,a). The output will have size [numsamples, 1] 
        """
        #Below is example code when neural network is set to take as input state and output Q-value for all actions.
        #This will be different for neural network that takes as input a state-action pair
        
        #q_preds = tf.cast(self.model(states), tf.double)
        #action_onehot = tf.cast(tf.one_hot(actions, self.actsize), tf.double)
        #q_preds_selected = tf.reduce_sum(q_preds * action_onehot, axis=-1)

        return q_preds_selected
        
    def compute_maxQvalues(self, states):
        """
        input: a list of numsamples states 
        output: max_a Q(s,a) values for every input state s in states. The output will have size numsamples
        """
        #Below is example code when neural network is set to take as input state and output Q-value for all actions.
        #if the neural takes as input a state-action pair, then the code will need to loop over all actions to compute all values
        
        #Qvalues = tf.cast(self.model(states), tf.double).numpy()
        #q_preds_greedy = np.max(Qvalues,1)
        
        return  q_preds_greedy
        
    def compute_argmaxQ(self, state):
        """
        input: one state s
        output: arg max_a Q(s,a) values for the input state s. The output will have size 1
        """
        #Below is example code when neural network is set to take as input state and output Q-value for all actions.
        #if the neural takes as input a state-action pair, then the code will need to loop over all actions to compute all values
        
        #values = tf.cast(self.model(state), tf.double).numpy()
        #greedy_action = np.argmax(values.flatten())
        
        return greedy_action
        
        
    def train(self, states, actions, targets):
        """
        states: numpy array as input to compute loss (s)
        actions: numpy array as input to compute loss (a)
        targets: numpy array as input to compute loss (Q targets)
        """
        with tf.GradientTape() as tape:
                        
            # COMPUTE Q PREDICTIONS for each state-action pair 
            q_preds_selected = self.compute_Qvalues(states, actions)
                      
            # CONSTRUCT LOSS FUNCTION as mean square error between  predicted $Q$-value and targets for each sample 
            #print(q_preds_selected.shape, targets.shape)
            loss = tf.reduce_mean((q_preds_selected - targets)**2)
            
            
            # BACKWARD PASS
            gradients = tape.gradient(loss, self.model.trainable_variables)  
        
            # UPDATE
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            
        return loss.numpy()

At this point you can skip ahead to implementing the basic Q-learning that at every step $t$ in the environment
* given state $s_t$, computes greedy actions from Q-values (using compute_argmaxQ function above) and uses $\epsilon$-greedy select an action $a_t$, 
* makes observation of reward $r_t$ and next state $s_{t+1}$
* using compute_maxQvalues() function, computes target
  $$r_t + \gamma \max_a Q_\theta(s_{t+1},a)$$
and then retrains the Q-function using train() function above (with numsamples=1)

However, for improved performance you may want to consider ideas like batch training (numsamples>1 is the batch size) with experience replay buffer and target-networks. 

**Replay Buffer**

Maintain a buffer $R$ to store trainsition tuples $(s_t,a_t,r_t,s_{t+1})$, when we minimize the Bellman error. When optimizing the Bellman error loss, we sample batches from the replay buffer and compute gradients for update on these batches. In particular, in each update, we sample $N$ tuples from buffer $(s_i,a_i,r_i,s_{i}') \sim R$ and then compute
targets

$$d_i=r_i + \max_a \gamma Q_{\theta}(s_i^\prime,a)$$
for all $i$. Use the above training function train() with input as list $(s_i, a_i, d_i)_{i=1}^N$  to update parameters using backprop.

**Target Network**

Maintain a target network in addition to the original pricipal network. The target network is just a copy of the original network but the parameters are not updated by gradients. The target network $\theta^-$ is copied from the principal network every $\tau$ time steps. Target network is used to compute the targets for update

$$d_i =  r_t + \gamma \max_a Q_{\theta^{-}}(s_{i}^\prime,a)$$

the targets are used in the loss function to update the principal network parameters. This slowly updated target network ensures that the targets come from a relatively stationary distribution and hence stabilize learning.

Hence several critical parts of the complete pseudocode for DQN is as follows:

**Initialization.**
principal network $Q_\theta(s,a)$, target network $Q_{\theta^{-}}(s,a)$. Replay buffer $R = \{\}$ (empty). 

**At each time step $t.$**
The agent executes action using $\epsilon-$greedy based on the principal network $Q_\theta(s,a)$. To update $\theta$: sample $N$ tuples $(s_i,a_i,r_i,s_i^\prime) \sim R$, compute empirical loss 

$$\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) - (r_i + \gamma \max_a Q_{\theta^{-}}(s_i^\prime,a))^2$$

and update parameter $\theta$ using backprop (just take one gradient step).

**Update target network.**
Every $\tau$ time steps, update target network by copying $\theta^{-} \leftarrow \theta$.

**Bellman target.**
Above, we have defined the target values as being computed from a target net with parameter $\theta^-$ 
$$r_i + \gamma \max_a Q_{\theta^{-}}(s_i^\prime,a)$$
It is worth thinking about what happens if we are at the end of an episode, that is, what if $s_i^\prime$ here is a terminal state. In this case, should the Bellman target be defined exactly the same as above? Do we need some modifications? Think carefully about this as this will greatly impact the algorithmic performance.

### Implementation of replay buffer 

In [ ]:
# Implement replay buffer
import random
class ReplayBuffer(object):
    
    def __init__(self, maxlength):
        """
        maxlength: max number of tuples to store in the buffer
        if there are more tuples than maxlength, pop out the oldest tuples
        """
        self.buffer = deque()
        self.number = 0
        self.maxlength = maxlength
    
    def append(self, experience):
        """
        this function implements appending new experience tuple
        experience: a tuple of the form (s,a,r,s^\prime)
        """
        self.buffer.append(experience)
        self.number += 1
        
    def pop(self):
        """
        pop out the oldest tuples if self.number > self.maxlength
        """
        while self.number > self.maxlength:
            self.buffer.popleft()
            self.number -= 1
    
    def sample(self, batchsize):
        """
        this function samples 'batchsize' experience tuples
        batchsize: size of the minibatch to be sampled
        return: a list of tuples of form (s,a,r,s^\prime)
        """
        minibatch = random.sample(self.buffer,batchsize)
        return minibatch
        

### Code snippet for copying target network
You may use th following to update target network i.e. to copy from principal network to target network. We need to use tensorflow scope to distinguish the computational graphs of target and principal networks. The following function builds a tensorflow operation that does the copying $\theta^- \leftarrow \theta$

In [ ]:
def run_target_update(Qprincipal, Qtarget):
    for v,v_ in zip(Qprincipal.model.trainable_variables, Qtarget.model.trainable_variables):
        v_.assign(v.numpy())

## Main code for DQN
Now that we have all the ingredients for DQN, we can write the main procedure to train DQN on a given environment. The implementation is straightforward if you follow the pseudocode pdf.


In [ ]:
%%wandb
#remove above line if you do not want to see inline plots from wandb

# hyper-parameters
lr = 1e-3  # learning rate for gradient update
batchsize = 64  # batchsize for buffer sampling
maxlength = 1000  # max number of tuples held by buffer
envname = "CartPole-v0"  # environment name
tau = 100  # time steps for target update
episodes = 200  # number of episodes to run
initialsize = 500  # initial time steps before start training
epsilon = .2  # constant for exploration
gamma = .99  # discount

# initialize environment
env = gym.make(envname)
obssize = env.observation_space.low.size
actsize = env.action_space.n

# initialize Q-function networks (princpal and target)
Qprincipal = Qfunction(obssize, actsize, lr)
Qtarget = Qfunction(obssize, actsize, lr)

# initialization of graph and buffer
buffer = ReplayBuffer(maxlength)

# main iteration
rrecord = []
totalstep = 0
for episode in range(episodes):
    
    obs = env.reset()
    done = False
    rsum = 0
    
    while not done:
        
        #greedy choice below. Use epsilon greedy for exploration
        action = Qprincipal.compute_argmaxQ(np.expand_dims(obs,0))
        
        newobs, r, done, _ = env.step(action)
        done_ = 1 if done else 0
        e = (obs, action, r, done_, newobs)

        #IF NOT USING BUFFER:
        #use single sample (obs, action, r, done_, newobs) with Qtarget to compute target and train Qprincipal
        
        # ELSE IF USING REPLAY BUFFER
        # append experiences e to buffer
        """
        buffer.append(e)
        while buffer.number > maxlength:
            buffer.pop()
        """
        #every few episodes (decide the frequency) sample a minibatch from buffer
        #compute targets in batch using Qtarget 
        #and train  Qprincipal 
        
        
        #UPDATE target network 
        #every tau steps update copy the principal network to the target network
        """
        if totalstep % tau == 0:
            run_target_update(Qprincipal, Qtarget)
        """
        
        # update
        totalstep += 1
        rsum += r
        obs = newobs 
        
    #Below are for printing and debugging at the end of episode
    rrecord.append(rsum)
    #if episode % 10 == 0:
        # Feel free to add printing functions for debugging purposes
        #print('buffersize {}'.format(buffer.number))
        #print('iteration {} ave training returns {}'.format(epsiode, np.mean(rrecord[-10:])))
        
    #printing moving averages for smoothed visualization. 
    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])
    wandb.log({ "training reward" : rsum, "train reward moving average" : movingAverage})
        

### Evaluation

Finally, we evaluate the performance of the trained agent. We will evaluate the performance of the greedy policy wrt learned Q-function. The evaluation will be run 10 times, each for eval_epsiodes and print out the average performance across these episodes. Please do not change the code below.

In [ ]:
### DO NOT CHANGE
def evaluate(Q, env, episodes):
    # main iteration
    
    for episode in range(episodes):
        
        obs = env.reset()
        done = False
        rsum = 0
        
        while not done:
            # always greedy
            action = Q.compute_argmaxQ(np.expand_dims(obs,0))
            

            # mdp stepping forward
            newobs, r, done, _ = env.step(action)

            # update data
            rsum += r
            obs = newobs        

        
        wandb.log({"eval reward" : rsum})

    return rsum

In [ ]:
# DO NOT CHANGE CODE HERE
# after training, we will evaluate the performance of the agent
# on a target environment
env_test = gym.make(envname)
eval_episodes = 300
score = np.mean([evaluate(Qprincipal, env_test, eval_episodes) for k in range(10)])
wandb.run.summary["score"]=score 

print("eval performance of DQN agent: {}".format(score))

In [ ]:
run.finish()